In [1]:
import math
import numpy as np
import pandas as pd
import ortools
import itertools

from typing import Generator, List, Tuple

In [2]:
# look at all possible cut assignments that result in the needed quantities
#   minimize over all of the cut assignments
from ortools.sat.python import cp_model


In [3]:
df = pd.read_csv("./#5 Consolidated.csv", header=None)
df.columns = ["length", "quantity"]

base_length = 240
waste_tolerance = 0

exact_cuts = df[df["length"] == base_length].copy()

# df = df[df["length"] < base_length]
needed_lengths = df["length"].values
needed_quantities = df["quantity"].values


# We expect to be allowed to make cuts
# 
# allowed_cuts = [
#   (3,), (3, 3,), (3, 3, 3,), (3, 5,), (5,), (5, 3,), (5, 5,), (8,),
# ]

In [4]:
len(df)

19

In [5]:
df["quantity"].sum()

976

In [6]:
df.sort_values("length", ascending=False)

,length,quantity
7,240,232
13,184,6
11,168,16
12,162,8
14,148,6
10,132,88
6,127,45
5,120,48
8,116,8
17,108,16


In [7]:
def get_possible_cuts(needed_lengths: List[int], base_length: int, tolerance_per_cut: float = 0) -> Generator:
    
    # Establish a bound on the number of combinations that we need
    max_combinations = math.floor(base_length / min(needed_lengths))

    for i in range(1, max_combinations + 1):
        for comb in itertools.combinations_with_replacement(needed_lengths, i):
            waste = len(comb) * tolerance_per_cut
            if len(comb) == 1 and comb[0] == base_length:
                yield comb
                continue
                
            if sum(comb) <= base_length - waste:
                yield comb

In [8]:
possible_cuts = list(
    get_possible_cuts(needed_lengths, base_length, tolerance_per_cut=.125)
)

In [9]:
len(possible_cuts)

584

In [10]:
cut_produced_quantities = {}
for i, cut in enumerate(possible_cuts):
    for j, length in enumerate(needed_lengths):
        cut_produced_quantities[i, j] = cut.count(length)
        
cut_wastes = {}
for i, cut in enumerate(possible_cuts):
    waste = base_length - sum(cut)    
    cut_wastes[i] = waste

In [11]:
model = cp_model.CpModel()
naive_max_cuts = sum(needed_quantities)

cut_vars = {}
for i, cut in enumerate(possible_cuts):
    cut_vars[i] = model.NewIntVar(0, naive_max_cuts, f"Cut according to rule {cut}")

for (j, length), needed in zip(enumerate(needed_lengths), needed_quantities): 
    cut_sums = []
    for i, cut in enumerate(possible_cuts):
        cut_sums.append(cut_vars[i] * cut_produced_quantities[i, j])
        
    model.Add(sum(cut_sums) >= needed)
    
# Let's minimize the cut wastest
total_cut_waste = sum(
    cut_vars[i] * cut_wastes[i] for i, _ in enumerate(possible_cuts)
)
model.Minimize(total_cut_waste)

In [12]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [13]:
print(solver.ResponseStats())

CpSolverResponse summary:
status: OPTIMAL
objective: 699
best_bound: 699
booleans: 600
conflicts: 65
branches: 1317
propagations: 817
integer_propagations: 38955
restarts: 4
lp_iterations: 0
walltime: 0.0898626
usertime: 0.0898629
deterministic_time: 0.00534172
gap_integral: 0.0362175



In [14]:
solution_df = []

for i, var in cut_vars.items():
#     if not solver.Value(var):
#         continue
    
    solution_df.append(
        {
            "cut": possible_cuts[i],
            "cut_idx": i,
            "number": solver.Value(var),
        }
    )
solution_df = pd.DataFrame(solution_df)
solution_df["waste"] = solution_df["cut_idx"].apply(cut_wastes.get)
solution_df["total_waste"] = solution_df["waste"] * solution_df["number"]

In [18]:
solution_df[solution_df["number"] > 0]

,cut,cut_idx,number,waste,total_waste
7,"(240,)",7,232,0,0
61,"(69, 168)",61,16,3,48
111,"(54, 184)",111,6,2,12
213,"(30, 162, 45)",213,8,3,24
235,"(66, 69, 103)",235,2,2,4
246,"(66, 127, 45)",246,45,2,90
257,"(66, 64, 108)",257,16,2,32
275,"(69, 116, 54)",275,8,1,8
302,"(96, 96, 45)",302,2,3,6
310,"(120, 54, 64)",310,48,2,96


In [21]:
total_waste = solution_df["total_waste"].sum()
total_length_needed = np.sum(df["length"] * df["quantity"])

In [29]:
print(f"We are {(1 - total_waste / total_length_needed)*100:.2f}% efficient")

We are 99.37% efficient
